In [1]:
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"
import sys
from utils.tools import load_obj, save_obj
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_squared_error, r2_score
from tqdm.notebook import tqdm
import os

In [2]:
def MAPE(pred, true):
    return np.mean(np.abs((pred - true) / true))
def cal_metrics(true, pred):
    rmse = np.sqrt(mean_squared_error(true, pred))
    r2 = r2_score(true, pred)
    mape = MAPE(pred, true)
    return {"rmse":rmse, "r2":r2, "mape":mape}

## nonformer-ewt 预测

In [3]:
N = 3
dataset = "toy_random_increase"
TARGET = "series"

In [4]:
S_nonformer_metrics_dict = {}
MS_nonformer_metrics_dict = {}
seq_len, pred_len = 50, 1
# total_len = len(df)
HORIZON = 10

nonformerNameDict = dict(zip(["lstm", "tpa", "tcn", "trans"], 
                         ["LSTM", "TPA", "TCN", "LogTrans"]))

tmp_nonformerNameDict = {}
tmp_nonformerNameDict.update(**nonformerNameDict)
tmp_nonformerNameDict.update(**dict(zip(["tpadecompose", "lstmdecompose", "tcndecompose", "transdecompose"],
                                       ["TPA-De", "LSTM-De", "TCN-De", "LogTrans-De"])))

In [5]:
for model_name in tmp_nonformerNameDict.keys():
    # 不同horizon下的分量预测结果
    ewt_metrics = {}
    trues_test_lst, preds_test_lst = [], []
    for hn in range(1, HORIZON+1):
        # 加载不同horizon下的各分量预测结果
        true_, pred_ = load_obj(f"./results/{dataset}/{model_name}_{dataset}_ftS_sl{seq_len}_ll0_pl{pred_len}_is1_os1_hn{hn}_bs32_lr0.001_{dataset}_0/true_pred.pkl")

        trues_test_lst.append(true_)
        preds_test_lst.append(pred_)
        ewt_metrics[f"horizon{hn}"] = cal_metrics(true_.flatten(), pred_.flatten())
        
    trues_test_all, preds_test_all = np.concatenate(trues_test_lst), np.concatenate(preds_test_lst)
    ewt_metrics["avghorizon"] = cal_metrics(trues_test_all.flatten(), preds_test_all.flatten())
    S_nonformer_metrics_dict[f"{tmp_nonformerNameDict.get(model_name)}"] = pd.DataFrame(ewt_metrics).T

In [6]:
# for model_name in tmp_nonformerNameDict.keys():
#     # 不同horizon下的分量预测结果
#     ewt_metrics = {}
#     trues_test_lst, preds_test_lst = [], []
#     for hn in range(1, HORIZON+1):
#         # 加载不同horizon下的各分量预测结果
#         true_, pred_ = load_obj(f"./results/{dataset}/{model_name}_{dataset}_ftMS_sl{seq_len}_ll0_pl{pred_len}_is4_os1_hn{hn}_bs32_lr0.001_{TARGET}multi.csv_0/true_pred.pkl")
#         trues_test_lst.append(true_)
#         preds_test_lst.append(pred_)
#         ewt_metrics[f"horizon{hn}"] = cal_metrics(true_.flatten(), pred_.flatten())
        
#     trues_test_all, preds_test_all = np.concatenate(trues_test_lst), np.concatenate(preds_test_lst)
#     ewt_metrics["avghorizon"] = cal_metrics(trues_test_all.flatten(), preds_test_all.flatten())
#     MS_nonformer_metrics_dict[f"{tmp_nonformerNameDict.get(model_name)}"] = pd.DataFrame(ewt_metrics).T

## AR

In [7]:
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA,ARIMA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

E:\ANACONDA\Lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
def get_result(val_data, test_data, rolling=True, pred_len=20, seq_len=200):
    history = [x for x in val_data[-seq_len:, 0]]

    preds_lst = list()
    trues_lst = list()
    initial = True
    for t in tqdm(range(len(test_data[:, 0])-pred_len)):
        history = history[-seq_len:]
        if initial:
            model = ARMA(history, order=(5, 0, 0))
            try:
                model_fit = model.fit()
            except:
                model_fit = model.fit(start_params=model_fit.params)
                
            # 仅估计一次模型，否则，滚动预测，迭代feed数据，重新估计模型
            if not rolling:
                initial = False
            
        output = model_fit.forecast(pred_len)
        yhat = output[0]
        preds_lst.append(yhat)
        obs = test_data[:, 0][t]
        history.append(obs)
        trues_lst.append(test_data[:, 0][t:t+pred_len])
    return preds_lst, trues_lst

In [9]:
# val_size = int(len(df)*0.1)
# test_size = int(len(df)*0.2)
# ar_data = df[cols].values
# train_data, val_data, test_data \
#     = ar_data[:(len(df)-test_size-val_size)], ar_data[(len(df)-test_size-val_size):(len(df)-test_size)], ar_data[-test_size:]

In [10]:
# arma_model=ARMA(train_data[:, 0], order=(5, 0, 0))
# arma_result=arma_model.fit()
# arma_result.summary()

In [11]:
# preds_lst, trues_lst = get_result(val_data, test_data, rolling=True)

In [12]:
# ar_preds = np.stack(preds_lst, axis=0)
# ar_trues = np.stack(trues_lst, axis=0)

# ar_metrics = {}
# for horizon in range(1, HORIZON+1):
#     ar_metrics[f"horizon{horizon}"] = cal_metrics(ar_trues[:, horizon-1], ar_preds[:, horizon-1])
# ar_metrics["avghorizon"] = cal_metrics(ar_trues.flatten(), ar_preds.flatten())
# metrics_dict["arima_rolling"] = pd.DataFrame(ar_metrics).T

In [13]:
# metrics_dict["arima_rolling"]

## arima static

In [14]:
# preds_lst, trues_lst = get_result(val_data, test_data, rolling=False)

In [15]:
# ar_preds = np.stack(preds_lst, axis=0)
# ar_trues = np.stack(trues_lst, axis=0)

# ar_metrics = {}
# for horizon in range(1, HORIZON+1):
#     ar_metrics[f"horizon{horizon}"] = cal_metrics(ar_trues[:, horizon-1], ar_preds[:, horizon-1])
# ar_metrics["avghorizon"] = cal_metrics(ar_trues.flatten(), ar_preds.flatten())
# metrics_dict["arima_static"] = pd.DataFrame(ar_metrics).T

In [16]:
# metrics_dict["arima_static"]

## 模型评估

In [17]:
def evaluation_show(metrics_dict, prefix, taget_indictor, target_horizon, sort=True, rotation=0):
    hist_dict = {}
    for key, value in metrics_dict.items():
        hist_dict[key] = value[taget_indictor][target_horizon]
    if sort:
        hist_dict = dict(sorted(hist_dict.items(), key=lambda x:x[1]))
    fig = plt.figure(figsize=(10, 6))
    x, width = 0, 0.25
    xtick_lst = []
    xlabel_lst = []
    for key, value in hist_dict.items():
        xtick_lst.append(x)
        xlabel_lst.append(key)
        plt.bar(x, value, width=width, label=key)
        x += width
    plt.xticks(xtick_lst, xlabel_lst, rotation=rotation)
    plt.title(f"{prefix}_{taget_indictor}_{target_horizon}")
    plt.legend(bbox_to_anchor=(1, 1));
    return fig, hist_dict

In [18]:
from pylab import mpl
import pylab
myparams = {
   'axes.labelsize': '15',
   'xtick.labelsize': '15',
    'lines.markersize' : 4.5,
   'ytick.labelsize': '15',
   'lines.linewidth': 2,
   'legend.fontsize': '15',
    'axes.titlesize': 15,
   'font.family': 'Times New Roman',
      'figure.figsize': '7,7',  #图片尺寸
    'mathtext.fontset':'stix',
    'axes.linewidth':1,
    'figure.dpi':300,
}
pylab.rcParams.update(myparams)  #更新自己的设置

In [19]:
features = "S"
modelNameDict = nonformerNameDict
MS_nonformer_metrics_dict = {}
if features == "MS":
    metrics_dict = MS_nonformer_metrics_dict
    rotation = 0
    cosumized = "MS"
else:
    metrics_dict = S_nonformer_metrics_dict
    rotation = 20
    cosumized = "S"
    
prefixDict = dict(zip(["toy_random_random", "toy_random_increase", "toy_stair_increase", "toy_stair_random"], 
                        ["toy_random_random", "toy_random_increase", "toy_stair_increase", "toy_stair_random"]))
prefix = prefixDict[dataset]

plot_metrics = metrics_dict
plot_rotation = rotation
plot_cosumized = cosumized

plot_all = True
if plot_all:
    plot_metrics_dict = {}
    plot_metrics_dict.update(**S_nonformer_metrics_dict)
    plot_metrics_dict.update(**MS_nonformer_metrics_dict)
    plot_cosumized = ""
    plot_rotation = 60

## horizon

In [20]:
def color_rank(df): 
    rank = df.rank()
    lst = []
    for i in rank:
        if i == 1:
            lst.append('color:red')
        elif i == 2:
            lst.append(f'color: green')
        else:
            lst.append(None)
    return lst

def bold_rank(df): 
    rank = df.rank()
    lst = []
    bold = "bold"
    for i in rank:
        if i <= 2:
            lst.append('font-weight: %s' % bold)
        else:
            lst.append(None)
    return lst
def count_performance1(df, model_lst):
    count = 0
    for col in model_lst:
        if df[f"{col}-De"]<df[f"{col}"]:
            count += 1
    return count
def count_performance2(df, model_lst):
    count = 0
    for col in model_lst:
        if (df[f"{col}-De"]<df[f"{col}"]) and (df[f"{col}-De"]<df[f"{col}-EWT"]):
            count += 1
    return count

def styler(df):
    cols = [col for col in df.columns if "Count" not in col]
    df = df.style.apply(func=color_rank, axis=1, subset=cols)
    df = df.apply(bold_rank, axis=1, subset=cols).format("{:.3f}", subset=cols)
    return df

In [21]:
save_metrics = {}
target_horizon_lst = ["horizon1", "horizon2", "horizon3", "horizon4", "horizon5",
                      "horizon6","horizon7","horizon8", "horizon9", "horizon10", "avghorizon"]#, "horizon6", "horizon10", "horizon20", "avghorizon"]
for key, value in metrics_dict.items():
    save_metrics[key] = metrics_dict[key].loc[target_horizon_lst, :]["rmse"].to_dict()

In [22]:
metrics_dict.keys()

dict_keys(['LSTM', 'TPA', 'TCN', 'LogTrans', 'TPA-De', 'LSTM-De', 'TCN-De', 'LogTrans-De'])

In [23]:
# MODEL_LST = set([i.split("-")[-1] for i in list(save_metrics.keys())])
MODEL_LST = modelNameDict.values()
save_metrics_df = pd.DataFrame(save_metrics).reindex(columns=sum([[i, f'{i}-De'] for i in modelNameDict.values()], []))
# save_metrics_df["Count"] = save_metrics_df.apply(count_performance, postfix1="De", postfix2="EWT", model_lst=MODEL_LST, axis=1)

save_metrics_df["Count1"] = save_metrics_df.apply(count_performance1, model_lst=MODEL_LST, axis=1)
# save_metrics_df["Count2"] = save_metrics_df.apply(count_performance2, model_lst=MODEL_LST, axis=1)
save_metrics_df = styler(save_metrics_df)
# save_metrics_df = save_metrics_df.style.apply(func=color_rank, axis=1, subset=save_metrics_df.columns[:-1])
# save_metrics_df = save_metrics_df.apply(bold_rank, axis=1, subset=save_metrics_df.columns[:-1]).format("{:.3f}", subset=save_metrics_df.columns[:-1])

In [24]:
# S
save_metrics_df

,LSTM,LSTM-De,TPA,TPA-De,TCN,TCN-De,LogTrans,LogTrans-De,Count1
horizon1,0.798,0.379,0.542,0.347,0.418,0.362,0.803,0.396,4
horizon2,0.852,0.350,0.540,0.351,0.419,0.374,1.022,0.404,4
horizon3,1.326,0.344,0.501,0.353,0.431,0.372,0.927,0.688,4
horizon4,0.941,0.346,0.547,0.353,0.411,0.373,0.906,0.371,4
horizon5,1.154,0.349,0.555,0.368,0.434,0.356,0.816,0.372,4
horizon6,0.810,0.348,1.101,0.356,0.464,0.359,0.847,0.371,4
horizon7,0.812,0.362,1.458,0.363,0.467,0.375,0.767,0.687,4
horizon8,0.988,0.368,0.527,0.382,0.495,0.389,0.840,0.450,4
horizon9,1.075,0.358,0.505,0.361,0.531,0.369,0.740,0.394,4
horizon10,1.227,0.395,2.103,0.357,0.469,0.381,0.862,0.402,4


In [26]:
postfix = ""
save_metrics_df.to_excel(f"./save_file/{prefix}_{cosumized}horizons{postfix}.xlsx")

## plot

In [27]:
# taget_indictor = 'rmse'
# target_horizon = "avghorizon"
# fig, hist_dict = evaluation_show(plot_metrics_dict, prefix, taget_indictor, target_horizon, rotation=plot_rotation)

In [28]:
# metrics_dict.update(**former_metrics_dict)

In [29]:
# fig.savefig(f"D:/汇报/Fig/{prefix}_{plot_cosumized}{taget_indictor}.pdf", bbox_inches="tight", dpi=300)

In [30]:
# taget_indictor = 'r2'
# fig, hist_dict = evaluation_show(plot_metrics_dict, prefix, taget_indictor, target_horizon, rotation=plot_rotation)

In [31]:
# fig.savefig(f"D:/汇报/Fig/{prefix}_{plot_cosumized}{taget_indictor}.pdf", bbox_inches="tight", dpi=300)

In [32]:
# taget_indictor = 'mape'
# fig, hist_dict = evaluation_show(plot_metrics_dict, prefix, taget_indictor, target_horizon, rotation=plot_rotation)

In [33]:
# fig.savefig(f"D:/汇报/Fig/{prefix}_{plot_cosumized}{taget_indictor}.pdf", bbox_inches="tight", dpi=300)